# Time Series

Стоковые данные для первого примера взяты [отсюда](https://www.kaggle.com/camnugent/sandp500)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Удобный способ читать временные данные, когда вы знаете, что столбец даты в нужном формате
df = pd.read_csv('../data/all_stocks_5yr.csv.zip', index_col='date', parse_dates=True)

In [ ]:
# df['Name'].unique()

In [ ]:
df = df[df['Name'] == 'GOOGL'].copy().drop('Name', axis=1)

In [ ]:
df.head()

In [ ]:
df['close'].plot(figsize=(15, 5))
plt.show()

Добавим скользящее среднее (moving average). Это можно сделать с помощью метода rolling, главное задать окну усреднения.  

In [ ]:
# 7 day rolling mean
df.rolling(7).mean().head(20)

In [ ]:
df['close_rolling_14'] = df['close'].rolling(window=14).mean()
df[['close','close_rolling_14']].plot(figsize=(15,5))
plt.show()

## Resample

Это приведение индекса к иному формату. К квартальному, годовому и прочим. Доступные форматы сожно посмотерть по [ссылке](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases).

Так как это, по сути, одна из разновидностей группировки, то надо задать математическую функцию, как объединять строки $(sum, mean, std, sem, max, min, median, first, last, ohlc)$.

In [ ]:
df.index

In [ ]:
# Средние квартальные данные
df.resample(rule='Q').mean()

In [ ]:
# Визуализируем данные по объему продаж акций по кварталам
df['volume'].resample('Q').mean().plot(kind='bar', figsize=(15, 5), color='b')
plt.title('Quarter volume')

## Time Shifting

Иногда необходимо сдвигать данные вперед или назад.

In [ ]:
# Вперед
df['open_1'] = df['open'].shift(1)

In [ ]:
df.head()

In [ ]:
# Назад
df['close_-1'] = df['close'].shift(-1)
df.head()

## Statsmodels
[Statsmodels](https://www.statsmodels.org/stable/index.html) - это питоновский модуль, который содержит реализацию различных статистических моделей, а также статистические тесты и способы исследования данных. 

In [ ]:
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('../data/all_stocks_5yr.csv.zip', index_col='date', parse_dates=True)
df = df[df['Name'] == 'GOOGL'].copy()
df.head()

### Поиск тренда
Фильтр Hodrick-Prescott разделяет временной ряд $y_t$  на тренд  $\tau_t$ и циклическую составляющую  $\zeta_t$

$y_t = \tau_t + \zeta_t$

In [ ]:
close_cycle, close_trend = sm.tsa.filters.hpfilter(df['close'])

In [ ]:
df['cl_cycle'] = close_cycle
df['cl_trend'] = close_trend

In [ ]:
df[['close', 'cl_trend']].plot(figsize=(15, 5))

### Поиск сезонной составляющей (Error, Trend, Seasonal)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df['close'], model='multiplicative', freq=30)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 10)

In [ ]:
df['cl_trend_ses'] = decomposition.trend

In [ ]:
df[['close', 'cl_trend', 'cl_trend_ses']].plot(figsize=(55, 5))

In [ ]:
df[['close', 'cl_trend', 'cl_trend_ses']]['2017-10-01':'2018-01-01'].plot(figsize=(15, 10))

## Exponentially-weighted moving average 

Выше мы разобрали работу простого скользящего окна. Но у этого метода есть недостатки:
* Малый размер окна ведет к большему шуму, чем сигналу;
* Лаг относительно размера окна;
* Некоторая потеря информаии из-за усреднения;
* Не дает понимания о поведении в будущем, показывает только существующие тренды;
* Экстримальные значения могут портить моделирование.

Модель эксопоненциально-взвешенного скользящего среднего может исправить некоторые недостатки прошлой модели (Exponentially-weighted moving average). Она придает больший вес значениям, которые были недавно, то есть в конце рассматриваемого периода. Значения весов зависят от размера окна и параметров модели.

[Полное описание модели можно найти зесь](http://pandas.pydata.org/pandas-docs/stable/computation.html#exponentially-weighted-windows)

Формула такова:

### $ y_t =   \frac{\sum\limits_{i=0}^t w_i x_{t-i}}{\sum\limits_{i=0}^t w_i} $

Где $x_t$ - это входное значение ряда, $w_i$ - вес, $y_t$ - выходное значение модели.

Если задать значение параметра adjust=True (default), то взвешенные средние будут считаться по следующей формуле:

### $y_t = \frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^t x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^t}$

Если adjust=False, то:

#### $\begin{split}y_0 &= x_0 \\
y_t &= (1 - \alpha) y_{t-1} + \alpha x_t,\end{split}$


Значение альфы можно задать несколькими способами:
 
* Span - аналог окна
* Center of mass, может быть выражен через span: c=(s−1)/2
* Период полураспада
* Напрямую заданная альфа.

Данные для дальнейшего примера [отсюда](https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/data)

In [ ]:
df = pd.read_csv('../data/air_visit_data.csv.zip', index_col='visit_date', parse_dates=True)
df = df[df['air_store_id'] == 'air_cb7467aed805e7fe']
df.drop('air_store_id', inplace=True, axis=1)
df.dropna(how='any', inplace=True)
df.head()

In [ ]:
df.plot(figsize=(15, 5))

In [ ]:
decomposition = seasonal_decompose(df, model='multiplicative', freq=30)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)

In [ ]:
df['ewma30'] = df.ewm(span=30).mean()
df['rol_30'] = df['visitors'].rolling(window=30).mean()

In [ ]:
df[['visitors','ewma30', 'rol_30']].plot(figsize=(35, 5))

# Arima (p,d,q)

## Autoregressive Integrated Moving Averages
Модель используются при работе с временными рядами для более глубокого понимания данных или предсказания будущих точек ряда.

### Параметры p,d,q

* p: Кол-во лагов;
* d: Порядок интегрированности;
* q: Размер окна, скользящего среднего.

Процесс подготовки и построения модели ARIMA
* Подготовка и визуализация данных
* Определение порядка интегрированности ряда и переход к стационарному ряду
* Построение и анализ автокорреляционной и частной автокорреляционной функции
* Построение модели
* Оценка работоспособности модели
* Прогнозирование с помощью модели


In [ ]:
df = pd.read_csv('../data/all_stocks_5yr.csv.zip', index_col='date', parse_dates=True)
df = df[df['Name'] == 'GOOGL'].copy()


In [ ]:
df.open.plot()

In [ ]:
decomposition = seasonal_decompose(df['open'], model='multiplicative', freq=365)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)

## Тест на стационарность

Для теста на стационарность можно использовать [Augmented Dickey-Fuller](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test) для поиска единичного корня ([unit root test](https://en.wikipedia.org/wiki/Unit_root_test)).

Нулевая гипотеза: единичный корень есть и ряд не стационарен.
Альтернативная гипотеза: ряд стационарен.

Решение можно принимать на основе p-value.

* Малое значение p-value (обычно ≤ 0.05) показывает сильные доказательства против нулевой гипотезы, значит, она отвергается в пользу альтернативной.

* Большое значение p-value (> 0.05) показывает слабые доказательства против нулевой гипотезы, значит, она не может быть опровергнута.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def adf_check(time_series):
    """
    На вход получает временной ряд, выдает ADF репорт
    """
    result = adfuller(time_series)
    print('Augmented Dickey-Fuller Test:')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print('------------------')
        print("Сильные доказательства против нулевой гипотезы, она отвергается. \nРяд не имеет единичного корня и является стационарным.")
    else:
        print('------------------')
        print("Слабые доказательства против нулевой гипотезы, она не отвергается. \nРяд имеет единичный корень и не является стационарным.")

In [ ]:
adf_check(df['open'])

## Определение порядка интегрированности ряда и переход к стационарному ряду

Порядок интегрированности - это сколько разностей вам надо взять, чтобы ряд стал стационарным.

In [ ]:
df['open_diff1'] = df['open'] - df['open'].shift(1)

In [ ]:
adf_check(df['open_diff1'].dropna())

In [ ]:
df['open_diff1'].plot()

# Построение и анализ автокорреляционной и частной автокорреляционной функции (ACF и PACF)

График автокорреляционной ф-ии показывает корреляцию ряда со своими лагами. По оси у - значение корреляции, по оси х - номер лага. По смыслу для стационарного процесса значение автокорреляции показывает, насколько в среднем изменится сегодняшний у, если у k-периодов назад, то есть yt-k, вырос на 1.

Эти графики строится по стационарным данным. Больше информации про ACF и PACF можно найти по ссылкам: [здесь](http://people.duke.edu/~rnau/arimrule.htm) и [здесь](https://people.duke.edu/~rnau/411arim3.htm).

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig_first = plot_acf(df["open_diff1"].dropna(), lags=40)

## Частная автокорреляционная функция

Частная корреляция — это обычная корреляция между yt и yt ‒ k, только yt и yt ‒ k должны быть очищены от влияния промежуточных случайных величин: yt ‒ 1, yt ‒ 2,..., yt ‒ k + 1.

In [ ]:
result = plot_pacf(df["open_diff1"].dropna(), lags =40)

## Интерпретация ACF и PACF

* Случай А. Процесс AR(p)
    1. ACF бесконечна по протяженности и только в пределе при k → ∞ сходится к нулю
    2. PACF равна (или близка) к нулю для лагов, больших, чем p.
* Случай Б. Процесс MA(q)
    1. ACF равна (или близка) к нулю для лагов, больших, чем q.
    2. PACF бесконечна по протяженности и только в пределе при k → ∞ сходится к нулю
* Случай В. Если не А и не Б, то у вас ARMA(p,q)

По возможности рекомендуется использовать экономичные модели: p + q ≤ 3 (если нет сезонной компоненты).

## Построение ARIMA модели

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(df['open'],order=(1,1,1))
results = model.fit()
print(results.summary())

## Оценивание
1. Значимость коэффициентов модели
2. Анализ остатков модели: 
    * Остатки должны быть белым шумом
    * Должны иметь нулевую автокорреляцию
    * Все элементы ACF для ряда остатков должны незначимо отличатся от нуля
3. Информационные критерии

In [ ]:
results.resid.plot()

In [ ]:
results.resid.plot(kind='kde')

In [ ]:
from statsmodels.graphics.api import qqplot

In [ ]:
fig_first = plot_acf(results.resid, lags=40)

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

In [ ]:
acorr_ljungbox(results.resid)

формально проверить автокорреляцию можно с помощью Ljungbox test.

Нулевая гипотеза: Автокорреляция отсутствует.
Альтернативная гипотеза: Автокорреляция присутствует.

Так как у нас p-value для всех рассмотренных лагов больше 0.05, то можно сделать вывод, что нулевая гипотеза не принимается и автокорреляция отсутствует.

## Предсказание

In [ ]:
import numpy as np
model = ARIMA(np.array(df['open'].dropna()), order=(1,1,1))
results = model.fit()
print(results.summary())

In [ ]:
res = results.plot_predict(start = df.shape[0]-300, end=df.shape[0], dynamic= True)

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,50) ]

In [ ]:
# future_dates

In [ ]:
df.index

In [ ]:
future_dates_df = pd.DataFrame(index=future_dates[1:],columns=df.columns)

In [ ]:
future_df = pd.concat([df,future_dates_df])

In [ ]:
future_df.tail()

In [ ]:
res = results.plot_predict(start = future_df.shape[0]-51, end=future_df.shape[0], dynamic= True)

In [ ]:
from sklearn.metrics import mean_squared_error
 
size = int((len(df)) * 0.95)
train, test = df['open'][0:size], df['open'][size:len(df)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(1,2,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
#     print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test RMSE: %.3f' % error**(1/2))
# plot
plt.plot([x for x in test])
plt.plot(predictions, color='red')
plt.show()

In [ ]:
predictions2 = [x for x in test[:-1]]

error = mean_squared_error(test[1:], predictions2)
print('Test RMSE: %.3f' % error**(1/2))

In [ ]:
plt.plot([x for x in test[1:]])
plt.plot(predictions2, color='red')

In [ ]:
error = mean_squared_error(test[1:], predictions[1:])
print('Test RMSE: %.3f' % error**(1/2))
# plot
plt.plot([x for x in test[1:]])
plt.plot(predictions[1:], color='red')
plt.show()